<a href="https://colab.research.google.com/github/LoolzMe/MachineLearning/blob/main/AIMLfC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

import urllib

In [2]:
(train_ds, test_ds, val_ds) = tfds.load('rock_paper_scissors', split=['train', 'test[70%:]', 'test[:30%]'], shuffle_files=True, as_supervised=True)

train_ds = train_ds.shuffle(1024).batch(2).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(1024).batch(2).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(1024).batch(2).prefetch(tf.data.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/2520 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteARJC6D/rock_paper_scissors-train.tfrec…

Generating test examples...:   0%|          | 0/372 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteARJC6D/rock_paper_scissors-test.tfreco…

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [3]:
weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

weights_file = "inception_v3.h5"

urllib.request.urlretrieve(weights_url, weights_file)

('inception_v3.h5', <http.client.HTTPMessage at 0x7fd0b6f7b2b0>)

In [4]:
inceptionv3 = InceptionV3(input_shape=(300, 300, 3), include_top=False, weights=None)

inceptionv3.load_weights(weights_file)

In [5]:
inceptionv3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [6]:
def freeze_model(model):
    for layer in model.layers:
        layer.trainable = False
    

In [7]:
freeze_model(inceptionv3)

In [8]:
IMG_SIZE = 300

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.2),
  layers.RandomTranslation(0.2, 0.2)
])

In [9]:
from tensorflow.python.ops.image_ops_impl import resize_images

big_model = Sequential([layers.InputLayer(input_shape=(300, 300, 3)), resize_and_rescale, data_augmentation])
big_model_inception = inceptionv3(big_model.output)

big_model = Model(big_model.input, big_model_inception)

output_layer = big_model.output

mini_model = layers.Flatten()(output_layer)
mini_model = layers.Dense(512, activation='selu')(mini_model)
mini_model = layers.Dropout(0.2)(mini_model)
mini_model = layers.Dense(256, activation='relu')(mini_model)
mini_model = layers.Dropout(0.2)(mini_model)
mini_model = layers.Dense(3, activation='softmax')(mini_model)

In [10]:
model = Model(big_model.input, mini_model)

model.compile(optimizer='rmsprop', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [17]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model", save_best_only=True, save_freq=600, monitor='accuracy')

In [19]:
with tf.device('/GPU:0'):
    model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[checkpoint])

Epoch 1/10
1199/1260 [===========================>..] - ETA: 3s - loss: 0.3811 - accuracy: 0.9783

1260/1260 [==============================] - 109s 85ms/step - loss: 0.3649 - accuracy: 0.9790 - val_loss: 12.6895 - val_accuracy: 0.7054
Epoch 2/10
 539/1260 [===========>..................] - ETA: 37s - loss: 0.3012 - accuracy: 0.9805

1260/1260 [==============================] - 105s 82ms/step - loss: 0.3542 - accuracy: 0.9782 - val_loss: 12.0723 - val_accuracy: 0.7411
Epoch 3/10
1260/1260 [==============================] - 68s 53ms/step - loss: 0.6052 - accuracy: 0.9687 - val_loss: 4.6003 - val_accuracy: 0.8661
Epoch 4/10
1260/1260 [==============================] - 68s 53ms/step - loss: 0.4583 - accuracy: 0.9742 - val_loss: 5.9482 - val_accuracy: 0.8571
Epoch 5/10
  34/1260 [..............................] - ETA: 1:04 - loss: 0.2114 - accuracy: 0.9706

KeyboardInterrupt: ignored

In [20]:
model.evaluate(test_ds)

56/56 [==============================] - 1s 20ms/step - loss: 5.7083 - accuracy: 0.8393


[5.7083306312561035, 0.8392857313156128]

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [22]:
!cp -R './best_model' /content/gdrive/MyDrive/AIMfC3/